In [7]:
import os
import numpy as np
import pandas as pd
import re

In [4]:
import warnings
warnings.simplefilter('ignore')
from statsbombpy import sb
sb.competitions().sort_values(['competition_id','season_id']).tail(5)  #this is a df and we get match id and season id from which

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
15,37,90,England,FA Women's Super League,female,False,False,2020/2021,2021-07-01T18:14:40.756,2021-06-13T16:17:31.694,None,2021-07-01T18:14:40.756
18,43,3,International,FIFA World Cup,male,False,True,2018,2021-12-06T23:11:59.985953,2021-06-13T16:17:31.694,None,2021-12-06T23:11:59.985953
36,49,3,United States of America,NWSL,female,False,False,2018,2021-11-06T05:53:29.435016,2021-06-13T16:17:31.694,None,2021-11-06T05:53:29.435016
38,55,43,Europe,UEFA Euro,male,False,True,2020,2022-02-01T17:20:34.319496,2021-11-11T13:54:37.507376,2021-11-11T13:54:37.507376,2022-02-01T17:20:34.319496
39,72,30,International,Women's World Cup,female,False,True,2019,2022-07-13T20:21:27.033445,2021-06-13T16:17:31.694,None,2022-07-13T20:21:27.033445


In [6]:
wc_matches=sb.matches(competition_id=43,season_id=3)
print(wc_matches.shape)
print(wc_matches.columns)
wc_matches.head(5)

(64, 22)
Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'match_status_360', 'last_updated', 'last_updated_360', 'match_week',
       'competition_stage', 'stadium', 'referee', 'home_managers',
       'away_managers', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version'],
      dtype='object')


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,7578,2018-06-15,14:00:00.000,International - FIFA World Cup,2018,Egypt,Uruguay,0,1,available,...,2021-06-13T16:17:31.694,1,Group Stage,\tEkaterinburg Arena,Björn Kuipers,Héctor Raúl Cúper,Óscar Washington Tabárez Silva,1.0.2,None,None
1,7553,2018-06-23,17:00:00.000,International - FIFA World Cup,2018,South Korea,Mexico,1,2,available,...,2021-06-13T16:17:31.694,2,Group Stage,\tRostov Arena,Milorad Mažić,Tae-Yong Shin,Juan Carlos Osorio Arbeláez,1.0.2,None,None
2,7544,2018-06-20,17:00:00.000,International - FIFA World Cup,2018,Uruguay,Saudi Arabia,1,0,available,...,2021-06-13T16:17:31.694,2,Group Stage,\tRostov Arena,Clément Turpin,Óscar Washington Tabárez Silva,Juan Antonio Pizzi Torroja,1.0.2,None,None
3,7536,2018-06-18,17:00:00.000,International - FIFA World Cup,2018,Belgium,Panama,3,0,available,...,2021-06-13T16:17:31.694,1,Group Stage,\tOlimpiyskiy Stadion Fisht,Janny Sikazwe,Roberto Martínez Montoliú,Hernán Darío Gómez Jaramillo,1.0.2,None,None
4,7555,2018-06-24,20:00:00.000,International - FIFA World Cup,2018,Poland,Colombia,0,3,available,...,2021-06-13T16:17:31.694,2,Group Stage,Kazan'' Arena,César Arturo Ramos Palazuelos,Adam Nawałka,José Néstor Pekerman,1.0.2,None,None


In [ ]:
# full_wc_2018_query
os.mkdir('fifa_wc_2018_statsbomb_query\fifa_wc_2018')
base_path='fifa_wc_2018_statsbomb_query\fifa_wc_2018'
teams=wc_matches.home_team.unique()

for team in teams:
    team_folder=os.path.join(base_path,team)
    lineup_folder=os.path.join(team_folder,'Team_Lineup')
    opp_lineup_folder=os.path.join(team_folder,'Opp_Lineup')
    events_folder=os.path.join(team_folder,'Match_Events')
    # opp_events_folder=os.path.join(team_folder,Opp_Events)
    
    os.mkdir(team_folder)
    os.mkdir(lineup_folder)
    os.mkdir(opp_lineup_folder)
    os.mkdir(events_folder)
    
    
    home = wc_matches[wc_matches['home_team']==team]
    away = wc_matches[wc_matches['away_team']==team]
    full=pd.concat([home,away],axis=0)
    full.sort_values('match_date')
    full.match_date = pd.to_datetime(full.match_date)
    full.sort_values('match_date',inplace=True)
    full.to_csv(os.path.join(team_folder,team+'-matches_player.csv'),header=True,index=False)
    
    
    for id in range(len(full.match_id)):
        match_id=full.match_id.iloc[id]
        lineup=sb.lineups(match_id=match_id)
        team_lineup=lineup[team]
        opp_team=[i for i in lineup if i !=team][0]
        opp_lineup=lineup[opp_team]
        
        date=str(full.iloc[id]['match_date'].date())
        home=str(team)[:3]
        away=str(opp_team)[:3]
        
        team_lineup.to_csv(os.path.join(lineup_folder,home+'-'+away+'-'+date+'.csv'),header=True,index=False)
        opp_lineup.to_csv(os.path.join(opp_lineup_folder,home+'-'+away+'-'+date+'.csv'),header=True,index=False)
        
        match_events=sb.events(match_id=match_id)
        # match_events=match_events[match_events['team']==team]
        # opp_match_events=match_events[match_events['team']==opp_team]

        mask_loc=match_events.location.notnull().values
        mask_pass_loc=match_events.pass_end_location.notnull().values

        match_events['loc_x']=match_events['location']
        match_events['loc_y']=match_events['location']
        match_events['loc_x'][mask_loc]=match_events[mask_loc].location.apply(lambda x : x[0])
        match_events['loc_y'][mask_loc]=match_events[mask_loc].location.apply(lambda x : x[1])

        match_events['p_loc_x']=match_events['pass_end_location']
        match_events['p_loc_y']=match_events['pass_end_location']
        match_events['p_loc_x'][mask_pass_loc]=match_events[mask_pass_loc].pass_end_location.apply(lambda x : x[0])
        match_events['p_loc_y'][mask_pass_loc]=match_events[mask_pass_loc].pass_end_location.apply(lambda x : x[1])
        match_events.drop(columns=['location','pass_end_location'],axis=1,inplace=True)
        match_events.to_csv(os.path.join(events_folder,home+'-'+away+'-'+date+'.csv'),header=True,index=False)